In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# plotly
import plotly.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go

#json
import json

# word cloud library
from wordcloud import WordCloud

# seabron
import seaborn as sns

# matplotlib
import matplotlib.pyplot as plt
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

# Gettin data

In [ ]:
data_movies = pd.read_csv('../input/tmdb_5000_movies.csv')
data_credits = pd.read_csv('../input/tmdb_5000_credits.csv')

# Glance over the both two data

In [ ]:
data_movies.info()

In [ ]:
data_credits.info()

# Coppying data 
Because of dont effect the for real data. Use this way we can use the coppy data like real. So I shorted this data for averages

In [ ]:
data_movies_average = data_movies .copy()
data_movies_average = data_movies_average.sort_values(['vote_average'],ascending = False)

# Just trying
its just my seaborn tryin. I just use seaborn lirary so this graphic is just for lesson.
But you can see real result under that.

In [ ]:
labelx = data_movies_average['original_title'].head(50)
labely = data_movies_average['vote_average'].head(50)

labelx = np.array(labelx)
labely = np.array(labely)

plt.figure(figsize= (18,12))
sns.barplot(x = labelx ,y = labely)
plt.xticks(rotation = 90)
plt.ylabel('Vote Average')
plt.xlabel('Title')

# Find Most Popular Movies
 
The first we should know scale of IMDB. How works IMDB? The scoring is defined by the average rating of the members.
However theese list are not baset on the average of votes of users. Because it is not healthy and fair result.For example, only two users voted for film A and average voted is 9 , 100 thousend users voted for film B and the average vote is 7,5 So if 100.000 people voted less then 2 people voted it would not be healthy.

Well thats why we use Bayesian Calculation Method. This method works like this:

 (v / (v+m)) x R + (m / (v+m)) x C
 
 R = vote average

 v = vote counts

 m = minimum votes required to be list

 C = average score for the entire report.
 
 So we have known alread R and V. We can find m and C by use the R and V

In [ ]:
C = data_movies.vote_average.mean()
print('the votes averages: ' ,C)

As you see the mean of vote oaverage is 6. and now we will find the minimum required votes. So firs we should determine a treshold. We will determined 93th percentile. Because for a movie be feature in the charts, it must have more votes than at least 90% of the movies in the list.But we will use higher than 90th percentile.

In [ ]:
m = data_movies.vote_count.quantile(.93)
print('the requred vote is : ',m)

Copying to data because of dont change everything in orginal data. After that make filter. We will choose high than m with use 'loc' method. Why we use loc method? Because m is not float which we found value.Thats wyh we cant comparison between m and the vote count. So we can select rows and columns based on labels with this method. The nice thing is that we can pass Boolean series instead of passing labels for a rows or columns selection and it will work.

In [ ]:
data_most_popular = data_movies.copy()


In [ ]:
data_most_popular = data_most_popular.loc[data_most_popular['vote_count']>=m]

In [ ]:
V = data_most_popular.vote_count
R = data_most_popular.vote_average

In [ ]:
result = (V/(V+m)*R) + (m / (V+m)* C)

We created new column in data and assigned to 'result' to the new column. After that we sorted the new column to the degree to 'result'.

In [ ]:
data_most_popular['pop'] = result

In [ ]:
data_most_popular

In [ ]:
data_most_popular = data_most_popular.sort_values('pop',ascending = False)

In [ ]:
data_most_popular[['title','vote_average']].head(10)

In [ ]:

data = [go.Bar(
        x=data_most_popular.title.head(50),
        y= data_most_popular['vote_average'].head(50),
        marker = dict(color = 'rgba(110, 0,255, 0.6)',
           line=dict(color='rgb(0,0,0)',width=1.5)),
            text = data_most_popular.title.head(50)
    )]

iplot(data, filename='basic-bar')

# Converting the data
Converted the data from json to string. Because we can use string easer.

In [ ]:
data_movies = pd.read_csv('../input/tmdb_5000_movies.csv')
data_credits = pd.read_csv('../input/tmdb_5000_credits.csv')
data_movies['genres']=data_movies['genres'].apply(json.loads)
for index,i   in zip(data_movies.index,data_movies['genres']):
    list1=[]    
    for j in range(len(i)):       
        list1.append((i[j]['name']))
    data_movies.loc[index,'genres'] = str(list1) 
    
data_movies['keywords']=data_movies['keywords'].apply(json.loads)
for index,i in zip(data_movies.index,data_movies['keywords']):
    list1=[]  
    for j in range(len(i)):
        list1.append((i[j]['name']))
    data_movies.loc[index,'keywords'] = str(list1) 
    
data_movies['production_companies'] = data_movies['production_companies'].apply(json.loads)
for index,i in zip(data_movies.index,data_movies['production_companies']):
    list1=[]
    for j in range(len(i)):
         list1.append((i[j]['name']))
    data_movies.loc[index,'production_companies'] = str(list1)     
    
data_movies['production_countries'] = data_movies['production_countries'].apply(json.loads)
for index,i in zip(data_movies.index,data_movies['production_countries']):
    list1=[]
    for j in range(len(i)):
        list1.append((i[j]['name']))
    data_movies.loc[index,'production_countries'] = str(list1)    
    
data_credits['cast'] = data_credits['cast'].apply(json.loads)
for index,i in zip(data_credits.index,data_credits['cast']):
    list1=[]
    for j in range(len(i)):
        list1.append((i[j]['name']))
    data_credits.loc[index,'cast'] = str(list1)     
    

# Cleaning the data
wehen we look see data we will see some strings.So this strings make complex to data. Thats why we need to clean this data.

In [ ]:
data_movies['genres'] = data_movies['genres'].str.strip('[]').str.replace(' ','').str.replace("''",'')
data_movies['genres'] = data_movies['genres'].str.split(',')

In [ ]:
list1 = []
for i in data_movies['genres']:
    list1.extend(i)

In [ ]:
most_genres = pd.Series(list1).value_counts().sort_values(ascending = False )
genres = most_genres.index
values = most_genres.values

In [ ]:
f,ax = plt.subplots(figsize = (12,12))
sns.barplot(x = values,y = genres,color = 'blue', alpha = .6)

# Cleaning data
We will see how many  movies made in each years. So thats why we need first cleaning to numerics data from than some strings. After that so many movies made same year but different month. Thats whay we need to first, cut up the months and days and counting againd the 'release_date'.

In [ ]:
data_movies.release_date = data_movies.release_date.apply(lambda x: x.replace('-',' ') if '-' in str(x) else x)
data_movies_release = data_movies.copy()
data_movies_release.release_date = data_movies_release.release_date.apply(lambda x: x[0:4] if len(str(x))>4 else str(x))
data_movies_release.release_date = data_movies_release.release_date.apply(lambda x: x.strip('nan') if 'nan' in str(x) else x)    
data_movies_release.release_date = pd.to_numeric(data_movies_release.release_date)  


In [ ]:
data_inf = data_movies_release.release_date.value_counts()

In [ ]:
labelx = np.array(data_inf.index)
labely = np.array(data_inf.values)

In [ ]:
plt.subplots(figsize = (18,15))
sns.pointplot(x=labelx,y=labely,color='blue',alpha = 0.1)
plt.xticks(rotation = 90)
plt.title('movies for years',fontsize = 20,color='red')
plt.xlabel('YEARS',fontsize = 20,color='green')
plt.ylabel('MOVİES',fontsize = 20,color='green')
plt.grid()

In [ ]:
g=sns.jointplot(labelx,labely,kind='kde',siz=20)
plt.savefig('graph.png')
plt.show()


In [ ]:
g = sns.jointplot(labelx,labely,size=7,color= 'blue',ratio = 3)
plt.show()

In [ ]:
genre_list = []
for index,row in data_movies.iterrows():
    genres = row['genres']
    for i in genres:
        genre_list.append(i)

  
    
   


In [ ]:
index = pd.Series(genre_list).value_counts().index
value = pd.Series(genre_list).value_counts().values

In [ ]:
list1=[]
for j in range(len(value)):
    list1.append(0)
   

In [ ]:
plt.subplots(figsize = (7,7))
colors = ['grey','blue','red','yellow','green','brown']
plt.pie(value,explode = list1,labels=index,autopct='%1.1f%%')
plt.show()

# Comparison of budget and revenue all movies. 





In [ ]:
df1=data_movies.copy()

In [ ]:
df1 = df1.sort_values('budget',ascending = False)

df1[['budget','genres','title','revenue']].head(10)

In [ ]:
list_budget = df1.budget.head(10)
list_revanue = df1.revenue.head(10)

In [ ]:
import plotly.graph_objs as go

trace1 = go.Bar (
                x = df1.title.head(10),
                y = list_budget,
                name = "Budget",
                marker = dict(color= 'rgba(0,0,200,0.4)',
                              line = dict(color = 'rgb(0,0,0)',width = 1)),
                text = df1.genres
              
               )


trace2 = go.Bar(
                x = df1.title.head(10),
                y = list_revanue,
                name = "revanue",
                  marker = dict(color= 'rgba(200,100,10,0.7)',
                                line= dict(color = 'rgb(0,0,0)',width = 1)),
                   text = df1.genres
)

data = [trace1, trace2]
layout = go.Layout(barmode='group')

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='grouped-bar')
    

# Finding the most productive countries and ratio, all over the world.
The firs step cleaning the data from strings and than we will create a list. 

In [ ]:
data_movies['production_countries'] = data_movies['production_countries'].str.strip('[]').str.replace(' ','').str.replace("''",'')
data_movies['production_countries'] = data_movies['production_countries'].str.split(',')

In [ ]:
list1 = []
for i in data_movies['production_countries']:
    list1.extend(i)
   


In [ ]:
df2 = pd.Series(list1).value_counts().dropna().head(10)
countries = df2.index
count = df2.values

In [ ]:
fig = {
    "data" : [
        {
            "values" : count,
            "labels" : countries,
            "domain" :{"x":[0,0.5]},
            "name" : "rate of product of countries",
            "hoverinfo": "label+percent+name",
            "hole": .3,
            "type": "pie"
            
            
        } ,],
    "layout":  {
        "title": "World rate of product of countries ",
        "annotations" : [
            {
                "font": {"size":20},
                "showarrow": False,
                "text": "rate of countries",
                "x": 0.20,
                "y": 1
            },
        ]
    }  
}
iplot(fig)